In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from acquire import *
import env
import json
from requests import get
from json.decoder import JSONDecodeError
from bs4 import BeautifulSoup
import time
from tqdm import tqdm

import env

In [2]:
"""
REPOS = [
    "gocodeup/codeup-setup-script",
    "gocodeup/movies-application",
    "torvalds/linux",
]
"""

'\nREPOS = [\n    "gocodeup/codeup-setup-script",\n    "gocodeup/movies-application",\n    "torvalds/linux",\n]\n'

In [3]:
links = pd.read_csv("links.csv", index_col="Unnamed: 0")

In [4]:
#When you get new links, delete this line.
links.href = links.href.str[1:]

In [5]:
links.head()

,href
0,learn-co-students/simple-blackjack-cli-prework
1,rocketacademy/basics-blackjack
2,datamllab/rlcard
3,learn-co-students/simple-blackjack-cli-001-pre...
4,cocos-creator/tutorial-blackjack-deprecated


In [6]:
#Run this to get new data
#df = get_repo_contents(links.href)

In [7]:
#Run to save the data
#df = pd.to_csv("origional_data.csv", index=False)

In [8]:
df = pd.read_csv("original_data.csv")

In [9]:
df.language.value_counts()

Ruby                216
Java                 52
JavaScript           40
Python               36
C#                   15
C++                   9
Swift                 7
HTML                  5
CSS                   5
TypeScript            4
Lua                   3
Objective-C           3
Jupyter Notebook      3
Vue                   2
Rust                  2
Kotlin                2
PHP                   2
Go                    2
Dart                  2
C                     1
Haskell               1
Visual Basic          1
Elixir                1
Roff                  1
Clojure               1
Solidity              1
Shell                 1
Name: language, dtype: int64